In [1]:
import numpy as np
import xarray as xr
from scipy.signal import butter, filtfilt

In [2]:
def butter_lowpass_filter_1d(y, cutoff, fs=1.0, order=4):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return filtfilt(b, a, y, axis=0)

In [3]:
da = xr.open_dataarray('data/line_208_interp.nc')

In [4]:
# convert to monthly mean (assumes time is datetime-like)
da_monthly = da.resample(date='1MS').mean(dim='date')  # '1MS' = start of month
# interpolate small gaps:
da_monthly_interp = da_monthly.interpolate_na(dim='date', method='linear', fill_value=np.nan)

In [5]:
# setting
cutoff = 1.0 / 12.0
fs = 1.0
order = 4

In [6]:
da_lowpass = xr.apply_ufunc(
    butter_lowpass_filter_1d,
    da_monthly,
    input_core_dims=[['date']],
    output_core_dims=[['date']],
    vectorize=True,
    kwargs={'cutoff': cutoff, 'fs': fs, 'order': order},
    dask='parallelized',
    output_dtypes=[da_monthly.dtype],
)

In [8]:
da_lowpass.to_netcdf('data/line_208_interp_lf.nc')